# Trading Strategy Backtests

A backtest of a trading strategy using the VectorBT library on tick data from January 2023 to May 2023.

In [ ]:
# Import libraries
import vectorbt as vbt
import pandas as pd
import numpy as np
import datetime
import talib
from numba import njit

In [ ]:
# Load and observe data. Make a timestamp as index
df = pd.read_csv('crypto_2023_upto_May8.csv')
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

df.head()

In [ ]:
# Check for NaN values
print(df.isna().value_counts())

In [ ]:
# Check data values
df.info()

In [ ]:
# shift close price by 10min compare it and place its value into classification equasion to get 'trend'
# 1 = long, -1 short, 0 equal.
df['ten_dir'] = np.where(df['close'] > df['close'].shift(10), 1,
                 np.where(df['close'] < df['close'].shift(10), -1, 0))


In [ ]:
# Remove first 10 rows that have no data
df = df[10:]

#### Extract the necessary value series from the dataframe.

In [ ]:
# Pull close price values
price = df.close
price.head()

In [ ]:
# Pull x_dist values
x_dist = df.x_dist
x_dist.head()

In [ ]:
# Volume values
vol = df.volume
vol.head()

In [ ]:
# Trend past 10 min
ten = df.ten_dir
ten.head()

### Create trading signal function


In [ ]:
# Make a function that finds outlier for n period with p percentile

def find_outlier(x_dist, n, p):
    # Create an array in a shape of x_dist with all False values
    outlier_flags = np.zeros_like(x_dist, dtype=bool)
    
    for i in range(n, len(x_dist)):
        start_index = i - n
        end_index = i
        period = x_dist[start_index:end_index]
        
        # Calculate the threshold based on the given percentile
        threshold = np.percentile(period, p)
        
        if x_dist[i] >= threshold:
            outlier_flags[i] = True
            
    return outlier_flags 

In [ ]:
# Function check
x = np.array([1,1,2,1,8,1,2,1,2,1,1,2,1,1,12])
n = 4
p = 75

outlier_flags = find_outlier(x, n, p)

print(outlier_flags)

### VectorBT setup for the multiparameter backtesting.
Trading logic function, custom indicators, parameters ranges and output charts.

In [ ]:
# RSI windows and n backtests

# Define RSI function using vbt wrap function and extracting RSI from talib
RSI = vbt.IndicatorFactory.from_talib('RSI')

# Define indicator and strategy
def optimize_x(price,x_dist, ten, vol,vol_val,window, n, p):
   
    # function
    outlier = find_outlier(x_dist,n,p)
    # Short Entries
    short = ((outlier == True) & (ten == 1) & (vol > vol_val))
    # Long entries
    long = ((outlier == True) & (ten == -1) & (vol > vol_val))
    
    #grab values of rsi
    rsi = RSI.run(price, window).real # .real means getting outcome values
    #Exits
    short_exit = (rsi <= 30)
    long_exit = (rsi >= 70)
    
    return long, long_exit, short, short_exit

# Create indicator function that uses defined strategy
x_ind = vbt.IndicatorFactory(
        class_name = 'optimizeX',
        short_name = 'x',
        input_names = ['price','x_dist', 'ten', 'vol'],
        param_names = ['vol_val','window', 'n', 'p'],
        output_names = ['entries', 'long_exit','short_entries','short_exit' ]
        ).from_apply_func(
                optimize_x,
                vol_val = 70000,
                window = 14,
                n = 120,
                p = 100,
                keep_pd=False)

# Create ranges of our tested parameters
ns = np.arange(300,800, step=20, dtype=int)
ps = np.arange(80,100, step=2, dtype=int)
windows = np.arange(12,20, step=1, dtype=int)
vols = np.arange(60000,200000, step=20000, dtype=int)

# Define results
res = x_ind.run(
        price,
        x_dist,
        ten,
        vol,
        vol_val=70000, 
        window = windows,
        n = ns,
        p = 100,
        param_product=True
        ) 

long_entries = res.entries
short_entries = res.short_entries
long_exits = res.long_exit
short_exits = res.short_exit


# Build portfolio
pf = vbt.Portfolio.from_signals(
    price,
    long_entries,
    long_exits,
    short_entries,
    short_exits,
    sl_stop=0.08,
    
    upon_dir_conflict = vbt.portfolio.enums.DirectionConflictMode.Ignore,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Reverse,
    freq='1T', fees = 0.0003)


returns = pf.total_return()
print(returns.max())
print(returns.idxmax())

In [ ]:
returns = returns.groupby(level=['x_n', 'x_window']).mean()
# Heatmap plot of input parameters
fig = pf.total_return().vbt.heatmap(
        x_level = 'x_window', # name of columns in printout of portfolio
        y_level = 'x_n',
        ) # slider option for heatmap to swith between symbols

fig.show()

In [ ]:
# Volume values and n backtests

# Define RSI function using vbt wrap function and extracting RSI from talib
RSI = vbt.IndicatorFactory.from_talib('RSI')

# Define indicator and strategy
def optimize_x(price,x_dist, ten, vol,vol_val,window, n, p):
   
    # function
    outlier = find_outlier(x_dist,n,p)
    # Short Entries
    short = ((outlier == True) & (ten == 1) & (vol > vol_val))
    # Long entries
    long = ((outlier == True) & (ten == -1) & (vol > vol_val))
    
    #grab values of rsi
    rsi = RSI.run(price, window).real # .real means getting outcome values
    #Exits
    short_exit = (rsi <= 30)
    long_exit = (rsi >= 70)
    
    return long, long_exit, short, short_exit

# Create indicator function that uses defined strategy
x_ind = vbt.IndicatorFactory(
        class_name = 'optimizeX',
        short_name = 'x',
        input_names = ['price','x_dist', 'ten', 'vol'],
        param_names = ['vol_val','window', 'n', 'p'],
        output_names = ['entries', 'long_exit','short_entries','short_exit' ]
        ).from_apply_func(
                optimize_x,
                vol_val = 70000,
                window = 14,
                n = 120,
                p = 100,
                keep_pd=False)

# Create ranges of our tested parameters
ns = np.arange(200,700, step=20, dtype=int)
ps = np.arange(80,100, step=2, dtype=int)
windows = np.arange(12,20, step=1, dtype=int)
vols = np.arange(60000,200000, step=20000, dtype=int)

# Define results
res = x_ind.run(
        price,
        x_dist,
        ten,
        vol,
        vol_val=vols, 
        window = 15,
        n = ns,
        p = 100,
        param_product=True
        ) 

long_entries = res.entries
short_entries = res.short_entries
long_exits = res.long_exit
short_exits = res.short_exit


# Build portfolio
pf = vbt.Portfolio.from_signals(
    price,
    long_entries,
    long_exits,
    short_entries,
    short_exits,
    sl_stop=0.08,
    
    upon_dir_conflict = vbt.portfolio.enums.DirectionConflictMode.Ignore,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Reverse,
    freq='1T', fees = 0.0003)


returns = pf.total_return()
print(returns.max())
print(returns.idxmax())

In [ ]:
returns = returns.groupby(level=['x_n', 'x_vol_val']).mean()
# Heatmap plot of input parameters
fig = pf.total_return().vbt.heatmap(
        x_level = 'x_vol_val', # name of columns in printout of portfolio
        y_level = 'x_n',
        ) # slider option for heatmap to swith between symbols

fig.show()

#### Best found parameters

In [ ]:
# Define RSI function using vbt wrap function and extracting RSI from talib
RSI = vbt.IndicatorFactory.from_talib('RSI')

# Define indicator and strategy
def optimize_x(price,x_dist, ten, vol,vol_val,window, n, p):
   
    # function
    outlier = find_outlier(x_dist,n,p)
    # Short Entries
    short = ((outlier == True) & (ten == 1) & (vol > vol_val))
    # Long entries
    long = ((outlier == True) & (ten == -1) & (vol > vol_val))
    
    #grab values of rsi
    rsi = RSI.run(price, window).real # .real means getting outcome values
    #Exits
    short_exit = (rsi <= 30)
    long_exit = (rsi >= 70)
    
    return long, long_exit, short, short_exit

# Create indicator function that uses defined strategy
x_ind = vbt.IndicatorFactory(
        class_name = 'optimizeX',
        short_name = 'x',
        input_names = ['price','x_dist', 'ten', 'vol'],
        param_names = ['vol_val','window', 'n', 'p'],
        output_names = ['entries', 'long_exit','short_entries','short_exit' ]
        ).from_apply_func(
                optimize_x,
                vol_val = 70000,
                window = 14,
                n = 120,
                p = 100,
                keep_pd=False)

# Create ranges of our tested parameters
ns = np.arange(640,800, step=20, dtype=int)
ps = np.arange(80,100, step=2, dtype=int)
windows = np.arange(12,20, step=1, dtype=int)
vols = np.arange(60000,200000, step=20000, dtype=int)

# Define results
res = x_ind.run(
        price,
        x_dist,
        ten,
        vol,
        vol_val=120000, 
        window = 15,
        n = 700,
        p = 100,
        param_product=True
        ) 

long_entries = res.entries
short_entries = res.short_entries
long_exits = res.long_exit
short_exits = res.short_exit


# Build portfolio
pf = vbt.Portfolio.from_signals(
    price,
    long_entries,
    long_exits,
    short_entries,
    short_exits,
    sl_stop=0.08,
    
    upon_dir_conflict = vbt.portfolio.enums.DirectionConflictMode.Ignore,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Reverse,
    freq='1T', fees = 0.0003)


returns = pf.total_return()
print(returns.max())
pf.plot().show()
pf.stats()

Many other conditions can be added, such as stop losses, take profits, trailing stops, different entry/exit criteria, etc., and tested quickly on large historical datasets.